#Homework 8 (10 pt)
<font color=red>**Danger zone:**</font> you'll be fine-tuning a model to generate positive, negative or even toxic reviews. We'll be doing this for fun, but this is also the technique for [review bombing](https://en.wikipedia.org/wiki/Review_bomb), bot farms on social media and other less than dignified stuff. It is ultimately your decision how you apply this knowledge, but before you choose, ask yourself: is this why you chose to learn ML?


# LLMs Alignment with Reinforcement Learning from human feedback (RLHF).

_based on the [notebook](https://github.com/yandexdataschool/nlp_course/blob/2024/week08_rlhf/practice.ipynb) based the [original notebook](https://github.com/antndlcrx/oxford-llms-workshop/blob/main/materials/seminars/day_3/8_LLMs%20alignment%20with%20RLHF.ipynb) by Ilya Boytsov for the Oxford LLMs workshop_



In this session, you're gonna fine-tune a language model with reinforcement learning to make it generate good (or bad) reviews.

To perform RL-based fine-tuning, we'll use a new (in this course) library called [Transformer Reinforcement Learning (TRL)](https://huggingface.co/docs/trl). TRL implements the main reinforcement learning components of RLHF: reward modeling and fine-tuning with PPO.

![img](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/TRL-readme.png)

In [1]:
#%pip install --quiet --upgrade transformers trl peft
#datasets-3.4.1 peft-0.15.0 transformers-4.50.1 trl-0.16.0 xxhash-3.5.0
# In the original notebook
#%pip install -q trl==0.7.4 transformers==4.33.1 datasets==2.14.4 peft==0.5.0

### Tutorial: align the model to generate positive movie reviews

To see how TRL works, we'll use it to align GPT2 on IMDB dataset to generate positive (or negative) movie reviews. In fact, __it's your choice whether you want positive or negative reviews.__

But before you choose, let's take a look at the baseline model: a GPT-2 fine-tuned on generating arbitrary movie reviews.

In [1]:
import torch
import transformers
device = torch.device('cuda')
main_tokenizer = transformers.AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb")
main_model = transformers.AutoModelForCausalLM.from_pretrained("lvwerra/gpt2-imdb", device_map=device)

/users/u28/spencer/anaconda3/envs/llm2/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/users/u28/spencer/anaconda3/envs/llm2/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/users/u28/spencer/anaconda3/envs/llm2/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/users/u28/spencer/anaconda3/envs/llm2/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils.

In [2]:
inputs = main_tokenizer("The movie", return_tensors='pt').to(device)
#generated_ids = main_model.generate(**inputs, max_new_tokens=50, do_sample=True)
#print("\nGenerated text:", main_tokenizer.decode(generated_ids.flatten().cpu().numpy().tolist()))

If you run this cell a couple of times, you'll see that the model generates both positive, negative and neutral reviews in some proportion. What we're gonna do next is teach the model to generate more positive (or negative) reviews.

Similarly to InstructGPT, we're gonna do that in 2 stages:
- **train a reward model** to assign higher values to positive (or negative) reviews
- fine-tune the language model to **maximize that reward using [proximal policy optimization](https://openai.com/research/openai-baselines-ppo)**



## Stage 1: train a reward model (2 points)

First, we'll train a BERT-like model as our reward model. We'll generate a synthetic pairwise rankings to emulate human rankings.

__Q:__ why do I need a reward model? Can I just use a pre-trained sentiment classifier? <br> __A:__ Yes, you can - but that only works for movie reviews. But this tutorial will teach you how to do RLHF for any kind objective.


__If you actually want to maximize sentiment (or other "label") instead of human preferences, train reward model as a classifier! (see week5)__


In [3]:
# We'll be fine-tuning a small BERT-like model for now. Please try other models for the main assignment.
reward_model = transformers.AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", device_map=device)
reward_tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-cased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


__Note that__ the reward model has a separate tokenizer, different from the main model. They don't need to be the same for RLHF fine-tuning.

In [4]:
# To train a reward model, you need a dataset (or generator) of positive-negative pairs.
# Each training sample should be a dict with 4 keys:
#  - input_ids_chosen, attention_mask_chosen = tokenizer("A sentence that human labeler likes more")
#  - input_ids_rejected, attention_mask_rejected = tokenizer("A sentence that human labeler likes less")

#import torch
import datasets

class IMDBPairwiseDataset(torch.utils.data.Dataset):
    """ A dataset of all possible pairs of chosen and rejected texts in TRT reward training format """
    def __init__(self, imdb, tokenizer, accepted_label: int):
        super().__init__()
        self.tokenizer = tokenizer
        self.chosen_texts = [row['text'] for row in imdb if row['label'] == accepted_label]
        self.rejected_texts = [row['text'] for row in imdb if row['label'] != accepted_label]

        assert self.chosen_texts, f"no texts with label {accepted_label}"

        print(f"Found {len(self.chosen_texts)} chosen and {len(self.rejected_texts)} rejected texts, {len(self)} pairs")

        # Add column_names attribute for compatibility
        self.column_names = [
            "input_ids_chosen",
            "attention_mask_chosen",
            "input_ids_rejected",
            "attention_mask_rejected"
        ]

    def __len__(self):
        return len(self.chosen_texts) * len(self.rejected_texts)

    def __getitem__(self, index: int):
        chosen = self.tokenizer(
            self.chosen_texts[index // len(self.chosen_texts)],
            truncation=True,
            padding="max_length",
            max_length=512,
            return_tensors="pt"
        )

        rejected = self.tokenizer(
            self.rejected_texts[index % len(self.rejected_texts)],
            truncation=True,
            padding="max_length",
            max_length=512,
            return_tensors="pt"
        )

        return {
            "input_ids_chosen": chosen["input_ids"].squeeze(),
            "attention_mask_chosen": chosen["attention_mask"].squeeze(),
            "input_ids_rejected": rejected["input_ids"].squeeze(),
            "attention_mask_rejected": rejected["attention_mask"].squeeze(),
        }


In [5]:
TARGET_LABEL = 0
imdb = datasets.load_dataset("imdb", split="train", download_mode="force_redownload")
reward_data = IMDBPairwiseDataset(imdb, reward_tokenizer, accepted_label=TARGET_LABEL)

sample = reward_data[31337]
print('CHOSEN:', reward_tokenizer.decode(sample['input_ids_chosen']))
print('REJECTED:', reward_tokenizer.decode(sample['input_ids_rejected']))

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Found 12500 chosen and 12500 rejected texts, 156250000 pairs
CHOSEN: [CLS] If only to avoid making this type of film in the future. This film is interesting as an experiment but tells no cogent story. < br / > < br / > One might feel virtuous for sitting thru it because it touches on so many IMPORTANT issues but it does so without any discernable motive. The viewer comes away with no new perspectives ( unless one comes up with one while one's mind wanders, as it will invariably do during this pointless film ). < br / > < br / > One might better spend one's time staring out a window at a tree growing. < br / > < br / > [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

We'll be using `trl.RewardTrainer` - a special case of `transformers.Trainer` that you used in the past. `RewardTrainer` accepts the same format of training arguments (e.g. batch size, gradient checkpointing) as before, except that it trains the model for the pairwise reward objective from [the InstructGPT paper](https://arxiv.org/pdf/2203.02155.pdf):

![img](https://i.imgur.com/2JzNAPs.png)

Note that the model itself does not score pairs: it processes chosen ($y_w$) and rejected ($y_l$) samples independently. To minimize this loss, the reward model needs to score chosen sample higher than the rejected one. Note that the formula also assumes some context $x$, which is useful for seq2seq tasks. In our case of movie reviews, $x$ is empty.

In [18]:
import datasets
from trl import RewardTrainer, RewardConfig
from transformers import DataCollatorWithPadding

# Define Dataset Class
class IMDBPairwiseDataset(torch.utils.data.Dataset):
    """ Creates a dataset of paired (chosen, rejected) texts for reward training. """
    def __init__(self, imdb, tokenizer, accepted_label: int):
        super().__init__()
        self.tokenizer = tokenizer
        self.chosen_texts = [row['text'] for row in imdb if row['label'] == accepted_label]
        self.rejected_texts = [row['text'] for row in imdb if row['label'] != accepted_label]

        assert self.chosen_texts, f"no texts with label {accepted_label}"
        print(f"Found {len(self.chosen_texts)} chosen and {len(self.rejected_texts)} rejected texts, {len(self)} pairs")

        # Add column_names attribute for RewardTrainer compatibility
        self.column_names = [
            "input_ids_chosen",
            "attention_mask_chosen",
            "input_ids_rejected",
            "attention_mask_rejected"
        ]

    def __len__(self):
        return min(len(self.chosen_texts), len(self.rejected_texts))

    def __getitem__(self, index: int):
        chosen_text = self.chosen_texts[index % len(self.chosen_texts)]
        rejected_text = self.rejected_texts[index % len(self.rejected_texts)]

        chosen_tokenized = self.tokenizer(chosen_text, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
        rejected_tokenized = self.tokenizer(rejected_text, truncation=True, padding="max_length", max_length=512, return_tensors="pt")

        return {
            "input_ids_chosen": chosen_tokenized["input_ids"].squeeze(),
            "attention_mask_chosen": chosen_tokenized["attention_mask"].squeeze(),
            "input_ids_rejected": rejected_tokenized["input_ids"].squeeze(),
            "attention_mask_rejected": rejected_tokenized["attention_mask"].squeeze(),
        }

# Create Dataset
TARGET_LABEL = 0  # Change to 0 for negative reviews
reward_data = IMDBPairwiseDataset(imdb, reward_tokenizer, accepted_label=TARGET_LABEL)
# Sample check
sample = reward_data[31337]
print('CHOSEN:', reward_tokenizer.decode(sample['input_ids_chosen']))
print('REJECTED:', reward_tokenizer.decode(sample['input_ids_rejected']))

Found 12500 chosen and 12500 rejected texts, 12500 pairs
CHOSEN: [CLS] I saw this movie in 1979, I was 17 or 18, when it was released. The theater was perhaps 1 / 4 full when the movie started. Ten minutes into the movie me and the friend who went with me to see the film were the only two people in the theater. The movie was really weird and had no plot or reason to its script and people demanded their money back. We decided to stay for the ENTIRE movie.... why endure such torture??... here's why. We wanted to be true movie critics... to have a standard to base all other movies on it is hard to justify saying you have seen the best movie ( a 10 ) they always come up with something better. But, it is easy to be able to base all other movies off of the worst movie ever made ( and this is it... a 1 at best ). There may be other movies out there that truly qualify as a 1, but I have yet to see them. I now base all movies I see on a scale based on this worst.... I AM A TRUE MOVIE CRITIC... 

In [7]:
# Define Data Collator (Handles Padding)
class RewardDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.default_collator = DataCollatorWithPadding(tokenizer)

    def __call__(self, batch):
        batch_chosen = [{"input_ids": ex["input_ids_chosen"], "attention_mask": ex["attention_mask_chosen"]} for ex in batch]
        batch_rejected = [{"input_ids": ex["input_ids_rejected"], "attention_mask": ex["attention_mask_rejected"]} for ex in batch]

        collated_chosen = self.default_collator(batch_chosen)
        collated_rejected = self.default_collator(batch_rejected)

        return {
            "input_ids_chosen": collated_chosen["input_ids"],
            "attention_mask_chosen": collated_chosen["attention_mask"],
            "input_ids_rejected": collated_rejected["input_ids"],
            "attention_mask_rejected": collated_rejected["attention_mask"],
        }

data_collator = RewardDataCollator(reward_tokenizer)

# Define Training Arguments
training_args = RewardConfig(
    output_dir="reward_model",
    remove_unused_columns=False,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    max_steps=1000,
    logging_steps=50,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    fp16=True if torch.cuda.is_available() else False,
    report_to="none"
)

# Train the Reward Model
trainer = RewardTrainer(
    model=reward_model,
    args=training_args,
    train_dataset=reward_data,
    data_collator=data_collator,  # Use custom collator instead of RewardDataProcessor
    peft_config=None
)

trainer.train()


/users/u28/spencer/anaconda3/envs/llm2/lib/python3.11/site-packages/accelerate/accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/users/u28/spencer/anaconda3/envs/llm2/lib/python3.11/site-packages/trl/trainer/reward_trainer.py:221: UserWarning: The current compute_loss is implemented for RewardDataCollatorWithPadding, if you are using a custom data collator make sure you know what you are doing or implement your own compute_loss method.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
50,0.567800
100,0.217900
150,0.152200
200,0.147400
250,0.128000
300,0.112200
350,0.117700
400,0.119800
450,0.080400
500,0.067200


Checkpoint destination directory reward_model/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/users/u28/spencer/anaconda3/envs/llm2/lib/python3.11/site-packages/trl/trainer/reward_trainer.py:221: UserWarning: The current compute_loss is implemented for RewardDataCollatorWithPadding, if you are using a custom data collator make sure you know what you are doing or implement your own compute_loss method.
  warnings.warn(
Checkpoint destination directory reward_model/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1000, training_loss=0.11633583378791809, metrics={'train_runtime': 542.2457, 'train_samples_per_second': 59.014, 'train_steps_per_second': 1.844, 'total_flos': 0.0, 'train_loss': 0.11633583378791809, 'epoch': 2.56})

In [8]:
reward_model.gradient_checkpointing_disable()
reward_model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### Sanity-check the reward model (1 point)

Let's check how our reward model performs.

__Your task__ is to measure how often does your reward model can rank a pair of (chosen and rejected) reviews correctly. Please measure this separately for train data (`imdb`) and a separate test set loaded below.

In [9]:
for sample_index in 45, 16000:
  print('TEXT:', imdb[sample_index]['text'])
  inputs = reward_tokenizer(
      imdb[sample_index]['text'], truncation=True, return_tensors='pt').to(device)
  with torch.no_grad():
    reward = reward_model(**inputs).logits[0, 0].item()
    print("REWARD:", reward)
  print('LABEL:', imdb[sample_index]['label'])
  print()

# note: your reward model may produce different absolute rewards.
# This is fine as long as the rewards are ordered correctly (most of the time)

TEXT: This movie sucked. It really was a waste of my life. The acting was atrocious, the plot completely implausible. Long, long story short, these people get "terrorized" by this pathetic "crazed killer", but completely fail to fight back in any manner. And this is after they take a raft on a camping trip, with no gear, and show up at a campsite that is already assembled and completely stocked with food and clothes and the daughters headphones. Additionally, after their boat goes missing, they panic that they're stuck in the woods, but then the daughters boyfriend just shows up and they apparently never consider that they could just hike out of the woods like he did to get to them. Like I said, this movie sucks. A complete joke. Don't let your girlfriend talk you into watching it.
REWARD: 4.57421875
LABEL: 0

TEXT: Good: Engaging cinematic firefights, great presentation, vehicles are actually fun to drive, fairly appealing multiplayer, faithful to the movie, and the list goes on.<br /

In [10]:
imdb_test = datasets.load_dataset("imdb", split='test')



In [11]:
imdb_test[24999]

{'text': 'I caught this movie on the Sci-Fi channel recently. It actually turned out to be pretty decent as far as B-list horror/suspense films go. Two guys (one naive and one loud mouthed a**) take a road trip to stop a wedding but have the worst possible luck when a maniac in a freaky, make-shift tank/truck hybrid decides to play cat-and-mouse with them. Things are further complicated when they pick up a ridiculously whorish hitchhiker. What makes this film unique is that the combination of comedy and terror actually work in this movie, unlike so many others. The two guys are likable enough and there are some good chase/suspense scenes. Nice pacing and comic timing make this movie more than passable for the horror/slasher buff. Definitely worth checking out.',
 'label': 1}

In [19]:
correct_predictions = 0
for sample_index in range(len(imdb_test)):
  inputs = reward_tokenizer(
      imdb_test[sample_index]['text'], truncation=True, return_tensors='pt').to(device)
  with torch.no_grad():
    reward = reward_model(**inputs).logits[0, 0].item()
#     print(reward, imdb_test[sample_index]['label'])
    if (reward > 0 and imdb_test[sample_index]['label'] == 0) or (reward <= 0 and imdb_test[sample_index]['label'] == 1):
        correct_predictions += 1
print('Accuracy on imdb_test = ', correct_predictions / len(imdb_test))


Accuracy on imdb_test =  0.91276


In [20]:
correct_predictions = 0
for sample_index in range(len(imdb)):
    inputs = reward_tokenizer(
      imdb[sample_index]['text'], truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        reward = reward_model(**inputs).logits[0, 0].item()
        if (reward > 0 and imdb[sample_index]['label'] == 0) or (reward <= 0 and imdb[sample_index]['label'] == 1):
            correct_predictions += 1
print('Accuracy on imdb_train = ', correct_predictions / len(imdb))

Accuracy on imdb_train =  0.92988


### Reward-guided generation (8 points)

If you did everything right, by now you should have a decent reward model. Before we use it for reinforcement learning, let's see if we can align model samples without any training.

To do so, you can use reward-guided inference: __generate N=16 samples, then select the one with the highest reward__ (according to your reward model).

For this problem, it's on you to demonstrate whether or not your code works. Find at least 5 neutral prompts such as "This movie is" (...), "It was" (...), "I'd like to say", generate samples, rank them based on reward and show which samples get the highest reward.

Note: it is faster to generate samples in parallel, rather than sequentially, as follows:




In [28]:
inputs = main_tokenizer(["It was"] * 5, return_tensors='pt').to(device)
for candidate in main_model.generate(**inputs, max_new_tokens=50, do_sample=True):
    print("Sample:", main_tokenizer.decode(candidate.flatten().cpu().numpy().tolist()))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sample: It was absolutely hilarious! He was able to carry it over to the real life. The story wasn't that bad either, but it was really sad. So for the time being, I'm just here for a great movie to remember.<br /><br
Sample: It was the most successful attempt of the American film industry at this, to break the status of Hollywood in the 1920s. There is no hint of the brilliance of the movie but the way it was distributed it was not an accident. The screenplay was written by
Sample: It was a pretty long movie about the history of human relations in this country, a history that is fascinating to watch today, and that gives a kind of insight into the nature of American politics and its roots, just where we have come from. This is the
Sample: It was very very good for a feature length film.<br /><br />The casting and the special effects could not keep up with the quality of the quality of the film itself.<br /><br />One of the scenes I just watched was the opening
Sample: It was very sad 

In [51]:
#8 neutral prompts so i can mult 2 on the imput
neutral_prompts = [
    "I watched the movie and it was",
    "My thoughts on the film: The movie was",
    "The cinematography in this film was",
    "I think the acting was",
    "The plot felt",
    "From a direction standpoint, it was",
    "Musically, the soundtrack was",
    "In terms of pacing, it was"
]

In [70]:
def generate_response(prompt):
    # 1. Assign eos as pad_token so generation doesn't error out
    main_tokenizer.pad_token = main_tokenizer.eos_token
    main_tokenizer.padding_side = "left"

    # 2. Tokenize with padding
    inputs = main_tokenizer(neutral_prompts * 8, return_tensors='pt', padding=True,).to(device)

    responses = []
    # 3. Generate with `pad_token_id` and `eos_token_id` BOTH set
    for candidate in main_model.generate(**inputs, max_new_tokens=100, do_sample=True,eos_token_id=main_tokenizer.eos_token_id,pad_token_id=main_tokenizer.eos_token_id):
        responses.append(main_tokenizer.decode(candidate.flatten().cpu().numpy().tolist()).replace(main_tokenizer.eos_token, ""))
    return responses

In [61]:
#calulate the score of the generated text
def get_best_responses(responses):
    inputs = reward_tokenizer(responses, truncation=True, padding=True, return_tensors='pt').to(device)
    scores = []
    for candidate in range(len(responses)):
        with torch.no_grad():
            reward = reward_model(**inputs).logits[candidate, 0].item()
            scores.append(reward)
    #sort scores and transfer the same sort to the responses
    sorted_scores = sorted(scores, reverse=True)
    sorted_responses = [x for _, x in sorted(zip(scores, responses), reverse=True)]
    return sorted_responses, sorted_scores

In [71]:
#generate the responses
responses = generate_response("The movie")
#calculate the scores
sorted_responses, sorted_scores = get_best_responses(responses)
# print the best response
print("Best response:", sorted_responses[0])


Best response: I think the acting was good. But i don't like my acting as a movie star. I don't know how i could get back to acting more in a 90's movie.<br /><br />i only felt bad in the movie. Maybe it was because i was a star of some sort, but the acting was just awful. The plot was nothing unique at all.<br /><br />This movie is a waste of time if you expect everything to work for you. Its not funny, its not scary


In [72]:
#lets run it 5 timed for fun
for i in range(5):
    responses = generate_response("The movie")
    sorted_responses, sorted_scores = get_best_responses(responses)
    print("Best response:", sorted_responses[0])
    print("Score:", sorted_scores[0])
    print()

Best response: My thoughts on the film: The movie was probably about one of the most stupid, crude, and/or grossing movies I've ever seen (which I'm sure has a lot more to its credit than this), where they go out of their way to make a bad movie that they're really not a bad movie. The script is so bad that that I can only imagine what a hell of a lot this means for the future of this genre.<br /><br />Seriously, I'm sure they could've done something nice about it
Score: 4.640625

Best response: In terms of pacing, it was really bad. The acting was terrible and the production was terrible. On a scale of 1 to 10, there was only a minor change to the plot but the main character was never seen. A 3.5 is a low end of the range of a low budget project produced by studio.<br /><br />If you have a good budget and budget, you can get an early picture. If you have a poor time to make a bad picture, don't waste your time by renting this one
Score: 4.59765625

Best response: My thoughts on the fi

# Fine-tune the main model with RL (FOR PROJECT)


For this tutorial, we will optimize GPT2 to produce positive IMDB movie reviews using the reward model you trained above.

Unlike supervised fine-tuning, RL allows model to generate it's own sentences on each training step. Then, it calculates the reward of those specific sentences, and finally, updates the model to increase the probability of sentences with high reward.

Thus, each RLHF consists of three stages: __Rollout__, __Evaluation__ and __Update__

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_bert_training.png' width='600'>

The update stage depends on the specific RL algorithm. We'll be using Proximal Policy Optimization, or [PPO](https://arxiv.org/abs/1707.06347), similarly to what was used for InstructGPT.

Before we run those 3 stages, however, we need to create a dataset of "queries" - partial reviews in our case.

In [ ]:
import trl

In [ ]:
import random

class LengthSampler:
    def __init__(self, min_length, max_length):
        self.min_length = min_length
        self.max_length = max_length

    def __call__(self):
        return random.randint(self.min_length, self.max_length)

sample_length = LengthSampler(2, 8)  # use the first 2-8 tokens as query


In [ ]:
def select_query_and_tokenize(sample):
    query_ids = main_tokenizer.encode(sample["text"])[: sample_length()]
    sample["query"] = main_tokenizer.decode(query_ids)  # query is the only required column
    sample["input_ids"] = query_ids  # to avoid re-tokenizing later
    return sample  # we do not need the rest - it will be generated by the model

imdb_for_rlhf = imdb.filter(lambda row: len(row['text']) > 200, batched=False)
imdb_for_rlhf = imdb_for_rlhf.remove_columns(['label'])
imdb_for_rlhf = imdb_for_rlhf.map(select_query_and_tokenize, batched=False)
imdb_for_rlhf.set_format(type="torch")

Next, let's prepare your reward model to predict rewards on whatever reviews were generated. Note that we use plaintext reviews because main model uses a different tokenizer from the reward model.

In [ ]:
#!g2.1
from typing import List
def compute_reward(texts: List[str]) -> torch.Tensor:
  inputs = reward_tokenizer(texts, truncation=True, padding=True, return_tensors='pt').to(device)
  with torch.no_grad():
    return reward_model(**inputs).logits[:, 0]

In [ ]:
#!g2.1
compute_reward([imdb[45]['text'], imdb[16000]['text']])  # test on human-written reviews

Finally, we move to RL training. In this tutorial, we'll train LoRA adapters and not the full model.

In [ ]:
#!g2.1
import peft
peft_config = peft.LoraConfig(
    task_type=peft.TaskType.CAUSAL_LM, r=32, lora_alpha=32, lora_dropout=0.0, inference_mode=False
)

# reload main model as AutoModelForCausalLMWithValueHead - with an extra head needed for PPO
main_tokenizer = transformers.AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb")
main_tokenizer.pad_token = main_tokenizer.eos_token

main_model = trl.AutoModelForCausalLMWithValueHead.from_pretrained("lvwerra/gpt2-imdb", device_map=device)
main_model = peft.get_peft_model(main_model, peft_config, adapter_name='default')
main_model.print_trainable_parameters()

Same as before, trl has a special type of trainer that minimize PPO-specific pseudo-loss. You can read more on this trainer [here](https://huggingface.co/docs/trl/main/en/ppo_trainer).

In [ ]:
#!g2.1
training_args = trl.PPOConfig(
    model_name=main_model.config._name_or_path,
    gradient_accumulation_steps=1,
    learning_rate=1.41e-5,
    batch_size=64,
    ppo_epochs=4,                 # PPO performs this many updates per training batch
)

ppo_trainer = trl.PPOTrainer(
    training_args, model=main_model.model, tokenizer=main_tokenizer,
    dataset=imdb_for_rlhf, data_collator=lambda data: dict((key, [d[key] for d in data]) for key in data[0])
)  # note: we pass main_model.model because PPOTrainer checks for one of several supported model types ...
# ... main_model.model is a model with adapters, which is supported. main_model itself is a wrapper that is not supported

In [ ]:
#!g2.1
from tqdm.auto import tqdm
max_steps = 50   # can be insufficient for some tasks - watch your learning curves
generation_kwargs = dict(
    min_length=-1, max_new_tokens=128, do_sample=True, top_k=0, top_p=1.0, pad_token_id=main_tokenizer.eos_token_id)
#                                  ^-- task-specific parameter!
with tqdm(enumerate(ppo_trainer.dataloader), total=max_steps) as progressbar:
  # note: ppo_trainer.dataloader is just a regular dataloader of queries, no RL-specific magic :)
  for epoch, batch in progressbar:
    if epoch >= max_steps:
        break

    # Rollout stage: generate continuations from batch queries using main_model
    response_tensors = ppo_trainer.generate(batch['input_ids'], **generation_kwargs)
    # ^-- list of tensors of token ids from main model tokenizer

    # de-tokenize responses to strings (since reward model uses a different tokenizer)
    batch["response"] = [main_tokenizer.decode(response.squeeze()) for response in response_tensors]
    # note: response_tensors already contain query tokens, so we don't need to add queries manually.
    # This may not be true for other tasks: check this manually by viewing batch["response"] and batch["query"]


    # Evaluation stage
    rewards = compute_reward(batch['response'])

    # Update stage
    stats = ppo_trainer.step(batch['input_ids'], response_tensors, list(rewards.split(1)))
    stats['rewards/mean'] = rewards.mean().item()

    print("-" * 30, 'STEP', epoch, '-' * 30)
    print(f'rewards/mean:\t{stats["rewards/mean"]:.9f}\t<---- average reward over this batch (higher=better, noisy)')
    print(f'ppo/returns/mean:\t{stats["ppo/returns/mean"]:.9f}\t<---- model-estimated average discounted reward')
    print(f'objective/kl:\t{stats["objective/kl"]:.9f}\t<---- how far we are from the original model (regularizer)')
    print()

    ppo_trainer.log_stats(stats, batch, list(rewards.split(1)))

In [ ]:
#!g2.1
print("-" * 30, 'STEP', epoch, '-' * 30)
print(f'rewards/mean:\t{stats["rewards/mean"]:.9f}\t<---- average reward over this batch (higher=better, noisy)')
print(f'ppo/returns/mean:\t{stats["ppo/returns/mean"]:.9f}\t<---- model-estimated average discounted reward')
print(f'objective/kl:\t{stats["objective/kl"]:.9f}\t<---- how far we are from the original model (regularizer)')
print()

## Main assignment - <u>actually</u> train the model


Your main task for this week is to use the RLHF pipeline to train a model for a reward of your choice. Here's what you can choose from:

__A. Toxicity fine-tuning:__ train the model to be less (or more!) toxic. For this task, you may use the data from [jigsaw toxic comments](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge) and [lmsys/toxic-chat](https://huggingface.co/datasets/lmsys/toxic-chat),  or any other source. Alternatively, you may use toxicity scores from [oasst1](https://huggingface.co/datasets/OpenAssistant/oasst1).


__B. Actual human feedback:__ use one of the existing datasets with pairwise human feedback to align your langauge model. You may use [anthropic's hh-rlhf](https://huggingface.co/datasets/Anthropic/hh-rlhf), [OpenAssistant dataset](https://huggingface.co/datasets/OpenAssistant/oasst1) or any other data you see fit. You may also turn the tables and train the model to [minimize](https://habrastorage.org/getpro/geektimes/post_images/ac7/2ad/827/ac72ad82767d4132164a4b6b76196c42.jpg) human preferences, as long as your model does not degrade to gibberish.

__C. Controlled generation:__ Instead of training a reward model from human feedback, you may define the reward function as the text length (longer or shorter) or number of times the model uses specific words (e.g. "sorry", "apologize"). If you choose specific words, make sure the model generates them at least sometimes.

__Alternatively,__ you may choose a different task. However, unless your task is very similar to one of the above, there is a chance that it will be **significantly** harder to solve, requiring orders of magnitude more compute and tuning. If you are in doubt, please ask the course staff. If they are AFK (again >.<), please prefer one of the recommended tasks.


#### General tips & tricks


Things to look out for:
- during PPO stage, the reward model should be in eval mode (dropout disabled)
- make sure max_length and max_new_tokens are enough for your chosen dataset - at least most of the time
- when in doubt, view the data manually or inspect how the model performs on a few samples


We highly recommend that you manually check the performance after each sub-stage:
1. when you assembled the pairwise dataset, inspect a couple of from of *your* dataset class and detokenize them. Make sure that you-the-human understand why one sample was accepted and the other - rejected. At least most of the time. This also lets you spot tokenization/truncation errors.
2. after you trained a reward model, measure how accurate this model is in isolation. If your reward model is poor, any subsequent RLHF will also fail.
3. once you've trained the main model with RL, ask it to generate examples and explore how well it does. If it produces an obviously bad output, check if the reward model assigns high reward to that output. If yes, reward model is the culprit; if no, it's a question of better/longer PPO training.

__It is also a good idea to periodically print samples during training.__

__When stuck, simplify the problem.__ If you've spent a several hours enchanting the reward model but it still won't budge, try switching to a simple subtask. For instance, if you're training on hh-rlhf, try limiting it the dataset to 10% of the shortest sequences - they are typically easier to learn.


## Assignment stages (and grading)

Regardless of the specific task you chose, your solution needs to contain several parts that will be graded separately.


#### Stage 1: reward model

Construct a dataset for training the reward model on your problem. Then, train a reward model on that dataset and evaluate how well can your model predict preferences on a hold-out (test) subset of your data.

Please make sure that the part of your notebook where you evaluate reward model is clearly visible and reasonably easy to read. And for all that is holy, do not call it IMDB unless it actually **is** data of imdb movie reviews :)

__Not all tasks require a reward model for later PPO fine-tuning.__ For instance, there's no reason to train a reward model if your reward equals sentence length. Likewise, toxicity reward can be estimated with a pre-trained toxicity classifier. __If your task does not require training a reward model, please train an unrelated model on [hh-rlhf](https://huggingface.co/datasets/Anthropic/hh-rlhf) as though you were solving assignment version B.__ This is for grading purposes only, you won't use this model for stage 2.


#### Stage 2: RL fine-tuning

Once the reward model is ready - or you can compute rewards without a model - it is time to maximize that reward with PPO. Optionally, you may replace PPO with another RL algorithm (or unlikelihood learning scheme), but only if you're feeling adventurous.


First, you need to choose a language model to be fine-tuned. You may choose any model, but make sure that your model **can** generate the data in your format. For instance, [Mistral-7B](https://huggingface.co/mistralai/Mistral-7B-v0.1) is a general purpose LM and may (or may not) need prompt engineering to generate chat assistant responses. For that reason, it is best if you **do not use `"lvwerra/gpt2-imdb"` unless you're generating only movie reviews**.



There are two "difficulty modes" for this task:
For the **easy mode**, use [gpt2-large](https://huggingface.co/gpt2-large) or [opt-1.3b](https://huggingface.co/facebook/opt-1.3b) with minimal code changes.
If you want the **Hard mode:** use a larger (e.g. 7B) model in combination with `load_in_4bit` and LoRA, the same way we did last week.
Some reasonable model choices are [LLaMA-7B](https://huggingface.co/Enoch/llama-7b-hf), [Falcon-7b](https://huggingface.co/tiiuae/falcon-7b), [Mistral-7B](https://huggingface.co/mistralai/Mistral-7B-v0.1) for general-purpose LM or [guanaco-7b](https://huggingface.co/timdettmers/guanaco-7b), [vicuna-7b](https://huggingface.co/lmsys/vicuna-7b-v1.5) for chat-based tasks, though there are many more (see [leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)). In the hard mode, you will need to modify the training arguments to enable 4-bit fine-tuning. Furthermore, your experiments will take somewhat longer to complete. On the plus side, your model will produce significantly better results.

__High reward is not enough!__ RL algorithms are famous for [cheating their reward functions](https://openai.com/research/faulty-reward-functions). To ensure that your model is actually doing what you want it to do, you will need some additional evaluation. To get the full grade, provide at least 20 side-by-side examples of your fine-tuned model vs original model predictions and a short summary.

Alternatively, you may provide 5 examples and some extrinsic evaluation metric over many examples. For instance, you may use a different pre-trained toxicity score for option A. When dealing with human preferences, you may choose to [enlist actual humans](https://toloka.ai/) or [ask GPT4/Claude](https://arxiv.org/pdf/2304.03277.pdf) to compare your model's predictions. For task C, when optimizing for simple rewards like sentence lengths, it is enough to compare histograms of rewards (e.g. average lengths).










